## Agents Env


### Agent Workspace Environment


Each agent work in ubuntu environment:

```dockerfile
FROM ubuntu:latest

RUN apt-get update

RUN apt install git -y

RUN apt install python3 -y

RUN apt install python3-pip -y

RUN mkdir -p /agent_workspace

WORKDIR /agent_workspace

CMD ["/bin/bash"]
```

Create docker image:

```bash
docker build -t agent_env .
```


Basic prompt for agents:

````python

prompt = """Act as {role}. Your responsibility is {responsibility}.

You have access to an Ubuntu environment, previously updated and located in your work folder.```
````


### Create Agent Env


In [2]:
import docker
import time

client = docker.from_env()
container_ref = None
CONTAINER_IMAGE = "agent_env"

def create_container(container_name):
    global container_ref

    existing_containers = client.containers.list(
        all=True, filters={"name": CONTAINER_NAME}
    )
    if existing_containers:
        container = existing_containers[0]
        if container.status != "running":
            container.start()
            time.sleep(20)
        container.reload()
        container_ref = container
        return "Container criado com sucesso."

    container = client.containers.run(
        CONTAINER_IMAGE,
        name=container_name,
        detach=True,
        tty=True,
        stdin_open=True,
        working_dir="/agent_workspace",
        command="/bin/bash",
    )

    time.sleep(20)

    container_ref = container
    return "Container criado com sucesso."

CONTAINER_NAME = "qa_agent_container"
create_container(CONTAINER_NAME)

'Container criado com sucesso.'

In [3]:
def delete_container(container_name):
    existing_containers = client.containers.list(
        all=True, filters={"name": container_name}
    )

    if existing_containers:
        container = existing_containers[0]
        container.stop()
        container.remove()
        return "Container deletado com sucesso."


# CONTAINER_NAME = "qa_agent_container"
# delete_container(CONTAINER_NAME)

In [4]:
def get_container():
    global container_ref

    if container_ref:
        container_ref.reload()
        if container_ref.status == "running":
            return container_ref

    containers = client.containers.list(all=True, filters={"name": CONTAINER_NAME})
    if containers:
        container = containers[0]
        if container.status != "running":
            container.start()
            time.sleep(60)
        container.reload()
        container_ref = container
        return container

    return None

### Utils function (get_container)


In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama

In [13]:
model_name = "llama3.1"
# model_name = "mistral-nemo"

model = ChatOllama(model=model_name)

### Run Command Tool (basic tool):


In [14]:
@tool
def terminal(command: str) -> str:
    """
    Execute a shell command to be executed.
    """
    container = get_container()
    if not container:
        return "No container found. Please try again."

    try:
        result = container.exec_run(f"/bin/sh -c '{command}'")
        return result.output.decode("utf-8")

    except Exception as e:
        return f"Error executing command: {str(e)}"

In [ ]:
tools = [terminal]
memory = MemorySaver()
agent_executor = create_react_agent(
    name="qa_agent",
    tools=tools,
    model=model,
)

description = """You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  
Follow the instructions carefully and execute the tasks precisely.  

System Information:
- OS: Ubuntu (root access available).
- Shell: Bash (no interactive commands allowed)."""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", description),
        ("user", "Perform the following tasks:\n\n{tasks}"),
    ]
)


tasks = """Create the 'projects' folder and clone the repository: https://github.com/astropy/astropy"""

prompt = prompt_template.invoke({"tasks": tasks})

response = agent_executor.invoke(prompt, {"recursion_limit": 100})
print(response)

TypeError: create_react_agent() got an unexpected keyword argument 'memory'

In [ ]:
tasks = """In 'projects/astropy' folder make checkout the specific commit: d16bfe05a744909de4b27f5875fe0d4ed41ce607"""

prompt = prompt_template.invoke({"tasks": tasks})

response = agent_executor.invoke(prompt, {"recursion_limit": 100})
print(response)

{'messages': [SystemMessage(content='You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  \nFollow the instructions carefully and execute the tasks precisely.  \n\nSystem Information:\n- OS: Ubuntu (root access available).\n- Shell: Bash (no interactive commands allowed).', additional_kwargs={}, response_metadata={}, id='5fe239e7-1a30-48dc-8b3f-c74275565e5b'), HumanMessage(content="Perform the following tasks:\n\nIn 'projects/astropy' folder make checkout the specific commit: d16bfe05a744909de4b27f5875fe0d4ed41ce607", additional_kwargs={}, response_metadata={}, id='d2802c7e-51e8-4df2-9e5b-cdc101a5a8c2'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-02-23T14:05:47.0225768Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1278174800, 'load_duration': 23674800, 'prompt_eval_count': 249, 'prompt_eval_duration': 258000000, 'eval_count': 46, 'eval_duration': 994000000, 'me

In [21]:
tasks = """In 'projects/astropy' folder create a virtual environment"""

prompt = prompt_template.invoke({"tasks": tasks})

response = agent_executor.invoke(prompt, {"recursion_limit": 100})
print(response)

{'messages': [SystemMessage(content='You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  \nFollow the instructions carefully and execute the tasks precisely.  \n\nSystem Information:\n- OS: Ubuntu (root access available).\n- Shell: Bash (no interactive commands allowed).', additional_kwargs={}, response_metadata={}, id='0111310a-dc03-4376-93f7-6a4fff8bc16a'), HumanMessage(content="Perform the following tasks:\n\nIn 'projects/astropy' folder create a virtual environment", additional_kwargs={}, response_metadata={}, id='b2e496ba-5854-4c3f-b5fd-1dde0df62e1f'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-02-23T14:09:05.5297302Z', 'done': True, 'done_reason': 'stop', 'total_duration': 885963300, 'load_duration': 21712900, 'prompt_eval_count': 224, 'prompt_eval_duration': 283000000, 'eval_count': 26, 'eval_duration': 578000000, 'message': Message(role='assistant', content='', im

In [25]:
tasks = """In 'projects/astropy' folder run pip install -r requirements.txt """

prompt = prompt_template.invoke({"tasks": tasks})

response = agent_executor.invoke(prompt, {"recursion_limit": 100})
print(response)

{'messages': [SystemMessage(content='You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  \nFollow the instructions carefully and execute the tasks precisely.  \n\nSystem Information:\n- OS: Ubuntu (root access available).\n- Shell: Bash (no interactive commands allowed).', additional_kwargs={}, response_metadata={}, id='8333951c-8e6c-403d-8aa7-8c65ef72b1a9'), HumanMessage(content="Perform the following tasks:\n\nIn 'projects/astropy' folder run pip install -r requirements.txt ", additional_kwargs={}, response_metadata={}, id='b3514ff8-dde9-415e-8e20-f700c4148243'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-02-23T14:14:31.8501395Z', 'done': True, 'done_reason': 'stop', 'total_duration': 979675900, 'load_duration': 22850000, 'prompt_eval_count': 228, 'prompt_eval_duration': 285000000, 'eval_count': 27, 'eval_duration': 670000000, 'message': Message(role='assistant', conten

In [ ]:
tasks = """1. Access the 'projects/astropy' and run tests pipeline.
2. Make a report of the test results."""

prompt = prompt_template.invoke({"tasks": tasks})

response = agent_executor.invoke(prompt, {"recursion_limit": 100})
print(response)

{'messages': [SystemMessage(content='You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  \nFollow the instructions carefully and execute the tasks precisely.  \n\nSystem Information:\n- OS: Ubuntu (root access available).\n- Shell: Bash (no interactive commands allowed).', additional_kwargs={}, response_metadata={}, id='795f215c-872e-4779-b343-3bb859c6c0bb'), HumanMessage(content="Perform the following tasks:\n\nSteps to follow:\n1. Access the 'projects/astropy' folder and install the requirements.\n2. Run the tests.", additional_kwargs={}, response_metadata={}, id='5d3ef649-834d-4ffb-9af6-bafaa9bc3d98'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral-nemo', 'created_at': '2025-02-19T17:32:55.4414162Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2606449700, 'load_duration': 22721100, 'prompt_eval_count': 142, 'prompt_eval_duration': 492000000, 'eval_count': 30, 'eval_duration': 20880000